In [1]:
import pandas as pd
import re
import ast
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/baraazekeria/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/baraazekeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/baraazekeria/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/baraazekeria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/baraazekeria/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv("data/super_scotus_cleaned.csv")

In [3]:
df

,votes_detail,win_side,title,petitioner,respondent,year,docket_no,justia_summary,justia_sections
0,"{'j__john_g_roberts_jr': 1.0, 'j__elena_kagan'...",0,Abbott v. United States,Kevin Abbott,United States,2010,09-479,"{'opinion_summary': [], 'primary_holding': []}","{'Opinion': ""(Ginsburg)': '\\n\\n\\nOPINION OF..."
1,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,"American Electric Power Co., Inc. v. Connecticut","American Electric Power Company Inc., et al.","Connecticut, et al.",2010,10-174,"{'opinion_summary': ['Plaintiffs, several stat...","{'Opinion': ""(Ginsburg)': '\\n\\n\\nOPINION OF..."
2,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,Arizona Christian School Tuition Organization ...,Arizona Christian School Tuition Organization,"Kathleen M. Winn, et al.",2010,09-987,"{'opinion_summary': [], 'primary_holding': []}","{'Opinion': ""(Kennedy)': '\\n\\n\\nOPINION OF ..."
3,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,Arizona Free Enterprise Club's Freedom Club PA...,Arizona Free Enterprise Club's Freedom Club PA...,"Ken Bennett, in His Official Capacity as Arizo...",2010,10-238,{'opinion_summary': ['The Arizona Citizens Cle...,"{'Opinion': '(Roberts)\': ""\\n\\n\\nOPINION OF..."
4,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,Ashcroft v. Al-Kidd,John Ashcroft,Abdullah al-Kidd,2010,10-98,"{'opinion_summary': [""Respondent alleged that,...","{'Opinion': ""(Scalia)': '\\n\\n\\nOPINION OF T..."
...,...,...,...,...,...,...,...,...,...
401,"{'j__john_g_roberts_jr': 1.0, 'j__anthony_m_ke...",1,Welch v. United States,Gregory Welch,United States,2015,15-6418,"{'opinion_summary': [""Possession of a firearm ...","{'Opinion': ""(Kennedy)': '\\n\\nNOTICE:\\u2002..."
402,"{'j__john_g_roberts_jr': 2.0, 'j__anthony_m_ke...",1,Whole Woman’s Health v. Hellerstedt,"Whole Woman’s Health, et al.","John Hellerstedt, Commissioner, Texas Departme...",2015,15-274,{'opinion_summary': ['Texas House Bill 2 (2013...,"{'Opinion': ""(Breyer)': '\\n\\nNOTICE:\\u2002T..."
403,"{'j__john_g_roberts_jr': 2.0, 'j__anthony_m_ke...",1,Williams v. Pennsylvania,Terrance Williams,Pennsylvania,2015,15-5040,{'opinion_summary': ['Williams was convicted o...,"{'Opinion': ""(Kennedy)': '\\n\\nNOTICE:\\u2002..."
404,"{'j__john_g_roberts_jr': 1.0, 'j__anthony_m_ke...",0,Wittman v. Personhuballah,"Robert J. Wittman, et al.","Gloria Personhuballah, et al.",2015,14-1504,"{'opinion_summary': [""Voters from Virginias Co...","{'Opinion': ""(Breyer)': '\\n\\nNOTICE:\\u2002T..."


In [4]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemmatization using NLTK
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
#     # Remove numbers
#     non_numeric_tokens = [word for word in lemmatized_tokens if not word.isnumeric()]
    
    # Remove words containing numbers
    non_numeric_tokens = [word for word in lemmatized_tokens if not re.search(r'\d', word)]
    
    # Convert to lowercase
    lowercase_tokens = [word.lower() for word in non_numeric_tokens]
    
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

In [5]:
df['justia_summary'] = df['justia_summary'].apply(preprocess_text)

In [6]:
df['justia_sections'] = df['justia_sections'].apply(preprocess_text)

In [7]:
assert not any(char.isdigit() for char in df['justia_summary']), "Numbers are present in the preprocessed text of the 'justia_summary' column."

In [8]:
assert not any(char.isdigit() for char in df['justia_sections']), "Numbers are present in the preprocessed text of the 'justia_summary' column."

In [9]:
df[['justia_summary', 'justia_sections']].head()

,justia_summary,justia_sections
0,"{ 'opinion_summary ' : [ ] , 'primary_holding ...",{ 'Opinion ' : `` ( Ginsburg ) ' : '\\n\\n\\nO...
1,"{ 'opinion_summary ' : [ 'Plaintiffs , several...",{ 'Opinion ' : `` ( Ginsburg ) ' : '\\n\\n\\nO...
2,"{ 'opinion_summary ' : [ ] , 'primary_holding ...",{ 'Opinion ' : `` ( Kennedy ) ' : '\\n\\n\\nOP...
3,{ 'opinion_summary ' : [ 'The Arizona Citizens...,{ 'Opinion ' : ' ( Roberts ) \ ' : `` \\n\\n\\...
4,{ 'opinion_summary ' : [ `` Respondent alleged...,{ 'Opinion ' : `` ( Scalia ) ' : '\\n\\n\\nOPI...


In [10]:
df.to_csv("data/super_scotus_text_preprocessed.csv", index = False)